In [1]:
# Starting with defining the network structure
from pgmpy.models import BayesianModel
cancer_model = BayesianModel([('Pollution', 'Cancer'),
('Smoker', 'Cancer'),
('Cancer', 'Xray'),
('Cancer', 'Dyspnoea')])
cancer_model.nodes()
cancer_model.edges()
cancer_model.get_cpds()

[]

In [2]:
# Now defining the parameters.
from pgmpy.factors.discrete import TabularCPD
cpd_poll = TabularCPD(variable='Pollution', variable_card=2,
values=[[0.9], [0.1]])
cpd_smoke = TabularCPD(variable='Smoker', variable_card=2,
values=[[0.3], [0.7]])
cpd_cancer = TabularCPD(variable='Cancer', variable_card=2,
values=[[0.03, 0.05, 0.001, 0.02],
[0.97, 0.95, 0.999, 0.98]],
evidence=['Smoker', 'Pollution'],
evidence_card=[2, 2])
cpd_xray = TabularCPD(variable='Xray', variable_card=2,
values=[[0.9, 0.2], [0.1, 0.8]],
evidence=['Cancer'], evidence_card=[2])
cpd_dysp = TabularCPD(variable='Dyspnoea', variable_card=2,
values=[[0.65, 0.3], [0.35, 0.7]],
evidence=['Cancer'], evidence_card=[2])

In [3]:
# Associating the parameters with the model structure.
cancer_model.add_cpds(cpd_poll, cpd_smoke, cpd_cancer, cpd_xray, cpd_dysp)
# Checking if the cpds are valid for the model.
cancer_model.check_model()
# Doing some simple queries on the network
cancer_model.is_active_trail('Pollution', 'Smoker')
cancer_model.is_active_trail('Pollution', 'Smoker', observed=['Cancer'])
cancer_model.get_cpds()
print(cancer_model.get_cpds('Pollution'))

+-------------+-----+
| Pollution_0 | 0.9 |
+-------------+-----+
| Pollution_1 | 0.1 |
+-------------+-----+


In [4]:
print(cancer_model.get_cpds('Smoker'))
print(cancer_model.get_cpds('Xray'))
print(cancer_model.get_cpds('Dyspnoea'))
print(cancer_model.get_cpds('Cancer'))

+----------+-----+
| Smoker_0 | 0.3 |
+----------+-----+
| Smoker_1 | 0.7 |
+----------+-----+
+--------+----------+----------+
| Cancer | Cancer_0 | Cancer_1 |
+--------+----------+----------+
| Xray_0 | 0.9      | 0.2      |
+--------+----------+----------+
| Xray_1 | 0.1      | 0.8      |
+--------+----------+----------+
+------------+----------+----------+
| Cancer     | Cancer_0 | Cancer_1 |
+------------+----------+----------+
| Dyspnoea_0 | 0.65     | 0.3      |
+------------+----------+----------+
| Dyspnoea_1 | 0.35     | 0.7      |
+------------+----------+----------+
+-----------+-------------+-------------+-------------+-------------+
| Smoker    | Smoker_0    | Smoker_0    | Smoker_1    | Smoker_1    |
+-----------+-------------+-------------+-------------+-------------+
| Pollution | Pollution_0 | Pollution_1 | Pollution_0 | Pollution_1 |
+-----------+-------------+-------------+-------------+-------------+
| Cancer_0  | 0.03        | 0.05        | 0.001       | 0.02     

In [5]:
cancer_model.local_independencies('Xray')
cancer_model.local_independencies('Pollution')
cancer_model.local_independencies('Smoker')
cancer_model.local_independencies('Dyspnoea')
cancer_model.local_independencies('Cancer')
cancer_model.get_independencies()

(Pollution _|_ Smoker)
(Pollution _|_ Xray, Dyspnoea | Cancer)
(Pollution _|_ Xray, Dyspnoea | Smoker, Cancer)
(Pollution _|_ Dyspnoea | Xray, Cancer)
(Pollution _|_ Xray | Dyspnoea, Cancer)
(Pollution _|_ Dyspnoea | Smoker, Xray, Cancer)
(Pollution _|_ Xray | Smoker, Dyspnoea, Cancer)
(Smoker _|_ Pollution)
(Smoker _|_ Xray, Dyspnoea | Cancer)
(Smoker _|_ Xray, Dyspnoea | Cancer, Pollution)
(Smoker _|_ Dyspnoea | Xray, Cancer)
(Smoker _|_ Xray | Dyspnoea, Cancer)
(Smoker _|_ Dyspnoea | Cancer, Xray, Pollution)
(Smoker _|_ Xray | Cancer, Dyspnoea, Pollution)
(Xray _|_ Smoker, Dyspnoea, Pollution | Cancer)
(Xray _|_ Smoker, Dyspnoea | Cancer, Pollution)
(Xray _|_ Dyspnoea, Pollution | Smoker, Cancer)
(Xray _|_ Smoker, Pollution | Dyspnoea, Cancer)
(Xray _|_ Dyspnoea | Cancer, Smoker, Pollution)
(Xray _|_ Smoker | Cancer, Dyspnoea, Pollution)
(Xray _|_ Pollution | Smoker, Dyspnoea, Cancer)
(Dyspnoea _|_ Smoker, Xray, Pollution | Cancer)
(Dyspnoea _|_ Smoker, Xray | Cancer, Pollution)
(Dy

In [7]:
# Doing exact inference using Variable Elimination
from pgmpy.inference import VariableElimination
cancer_infer = VariableElimination(cancer_model)
# Computing the probability of bronc given smoke.
q = cancer_infer.query(variables=['Cancer'], evidence={'Smoker': 1})
print(q['Cancer'])
# Computing the probability of bronc given smoke.
q = cancer_infer.query(variables=['Cancer'], evidence={'Smoker': 1})
print(q['Cancer'])
# Computing the probability of bronc given smoke.
q = cancer_infer.query(variables=['Cancer'], evidence={'Smoker': 1,'Pollution': 1})
print(q['Cancer'])

+----------+---------------+
| Cancer   |   phi(Cancer) |
+==========+===============+
| Cancer_0 |        0.0029 |
+----------+---------------+
| Cancer_1 |        0.9971 |
+----------+---------------+
+----------+---------------+
| Cancer   |   phi(Cancer) |
+==========+===============+
| Cancer_0 |        0.0029 |
+----------+---------------+
| Cancer_1 |        0.9971 |
+----------+---------------+
+----------+---------------+
| Cancer   |   phi(Cancer) |
+==========+===============+
| Cancer_0 |        0.0200 |
+----------+---------------+
| Cancer_1 |        0.9800 |
+----------+---------------+


/home/cmruuser/anaconda3/lib/python3.7/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:598: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi1.values = phi1.values[slice_]
